If we use all data, Batch(improve the speed of training)
If we use only one sample(stochastic gradient descenet)
Using mini-batch to balance this two

Epochs: One forward pass and one backward pass of all the training example
Batch-size: The number of training examples in one forward backward pass
Iteration: Number of passes, each pass using `batch size` number of examples
```{python}
# Training cycle
for epoch in range(training_epochs):
    # Loop over all batches
    for i in range(total_batch):
```

### DataLoader: batch_size=2, shuffle=True
As long as dataset provides index, and length
1. shuffle the dataset
2. loader the dataset, turn it into iterable loader

In [5]:
import torch.nn.functional as F
import torch
import numpy as np
from torch.utils.data import Dataset
# Dataset is an abstract class
from torch.utils.data import DataLoader
class Model(torch.nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super(Model, self).__init__(*args, **kwargs)
        self.linear1 = torch.nn.Linear(8, 6)
        self.linear2 = torch.nn.Linear(6, 4)
        self.linear3 = torch.nn.Linear(4, 1)
        self.sigmoid = torch.nn.Sigmoid() ## 

    def forward(self, x):
        x = self.sigmoid(self.linear1(x))
        x = self.sigmoid(self.linear2(x))
        x = self.sigmoid(self.linear3(x))
        return x

In [7]:
class DiabetesDataset(Dataset):
    def __init__(self, filepath):
        xy = np.loadtxt(filepath, delimiter=',', dtype=np.float32)
        self.len = xy.shape[0]
        self.x_data = torch.from_numpy(xy[:, :-1])
        self.y_data = torch.from_numpy(xy[:, [-1]])
    
    
    # magic function
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    
    # this magic function returns length of dataset
    def __len__(self):
        return self.len

model = Model()
criterion = torch.nn.BCELoss(size_average=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

dataset = DiabetesDataset('diabetes.csv.gz')
# initialize loader with batch-size, shuffle, process_number
train_loader = DataLoader(dataset=dataset,
                          batch_size=32,
                          shuffle=True)

for epoch in range(100):
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        y_pred = model(inputs)
        loss = criterion(y_pred, labels)
        
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
        